In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import cactas as C

2023-01-28 21:58:26.713840: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [3]:
model = C.Helper.load_unet('/raid/psych/CACTAS/unet_full_33_cases_weights.hdf5')

2023-01-28 21:58:28.154547: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-01-28 21:58:28.242733: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-01-28 21:58:28.242784: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-01-28 21:58:28.246145: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-01-28 21:58:28.246182: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-01-28 21:58:28.247094: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10
2

In [4]:
TEST_DATA = '/raid/mpsych/CACTAS/DATA/Nathan_Test/'
TEST_DATA_OUT = '/raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/'
TEST_DATA_IGNORE = ['41','51','61'] # z-spacing different

In [5]:
import os
TEST_SCANS = sorted(os.listdir(TEST_DATA))

In [6]:
THRESHOLD = .5

In [11]:
JACCARDS = {}
TIMINGS = {}

for i,s in enumerate(TEST_SCANS):
    
    s_stripped = s.replace('ESUS ','')
    
    if s_stripped in TEST_DATA_IGNORE:
        continue 

    image, label = C.Util.load(i, datapath=TEST_DATA)
    

    label_pred = np.zeros(label.shape, dtype=bool)
    label_pred[:,:,:] = 0
    
    jaccard_per_slice = []
    total_time = 0
    
    for z in range(image.shape[2]):
        
        t0 = time.time()

        cur_slice = image[:,:,z]

        cur_slice = C.Util.normalize(cur_slice)

        cur_slice_pred = model.predict(cur_slice.reshape(1,512,512,1))

        cur_slice_pred = C.Util.binarize(cur_slice_pred[0,:,:,0], threshold=THRESHOLD)

        label_pred[:,:,z] = cur_slice_pred

        total_time += time.time()-t0
        
        jac = C.Util.jaccard(C.Util.binarize(label[:,:,z]), cur_slice_pred)

        jaccard_per_slice.append(jac)
        
    TIMINGS[s] = total_time
    JACCARDS[s] = jaccard_per_slice
    print(s, np.mean(jaccard_per_slice), np.std(jaccard_per_slice))
    
    np.save(TEST_DATA_OUT+s_stripped+'.npy', label_pred)
    print('Stored', TEST_DATA_OUT+s_stripped+'.npy')

ESUS 10 0.39688863082513465 0.4834835972139254
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/10.npy
ESUS 14 0.2901028174685464 0.4509860247192325
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/14.npy
ESUS 15 0.4275377566075241 0.49023965904928707
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/15.npy
ESUS 2 0.4420303654015244 0.4943147231933135
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/2.npy
ESUS 27 0.5089031339031339 0.49749004247244394
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/27.npy
ESUS 32 0.4323567404122562 0.4925183177730497
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/32.npy
ESUS 33 0.37327829511535815 0.48257434841509056
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/33.npy
ESUS 35 0.33646604441583516 0.47111666171402267
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/35.npy
ESUS 39 0.33954339101199865 0.4690247219488885
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/39.npy
ESUS 46 0.42127659574468085 0.4

In [12]:
TIMINGS

{'ESUS 10': 12.387959957122803,
 'ESUS 14': 13.235450506210327,
 'ESUS 15': 8.907161235809326,
 'ESUS 2': 12.771570444107056,
 'ESUS 27': 11.552951097488403,
 'ESUS 32': 12.527665853500366,
 'ESUS 33': 13.451958417892456,
 'ESUS 35': 12.484254598617554,
 'ESUS 39': 12.716676712036133,
 'ESUS 46': 12.44057559967041,
 'ESUS 47': 9.943318128585815,
 'ESUS 48': 27.496068000793457,
 'ESUS 5': 10.398661375045776,
 'ESUS 50': 12.668991088867188,
 'ESUS 52': 12.261818170547485,
 'ESUS 53': 12.46430230140686,
 'ESUS 57': 12.268943071365356,
 'ESUS 64': 12.526496648788452,
 'ESUS 66': 12.573646783828735,
 'ESUS 73': 13.261152982711792,
 'ESUS 75': 12.671327114105225,
 'ESUS 83': 13.855017900466919,
 'ESUS 84': 8.763141632080078,
 'ESUS 86': 12.309324264526367,
 'ESUS 9': 12.549145460128784,
 'ESUS 91': 9.030827283859253,
 'ESUS 93': 11.338948488235474,
 'ESUS 94': 10.420567035675049}